## Imports

In [178]:
import numpy as np
from qutip import about, basis, rand_ket, tensor
from qutip_qip.operations import Measurement
from qutip_qip.operations.gates import gate_sequence_product
from qutip_qip.qasm import read_qasm, print_qasm

Random circuits: https://docs.quantum.ibm.com/api/qiskit/circuit_random#random-circuits

In [179]:
#make a random quantum circuit with qiskit

from qiskit.circuit.random import random_clifford_circuit
 
qiskit_circ = random_clifford_circuit(num_qubits=3, num_gates=20)
qiskit_circ.draw()

┌───┐┌───┐                                   ┌───┐┌──────┐          ┌───┐
q_0: ┤ Y ├┤ X ├────────────────────────────────■──┤ Z ├┤ √Xdg ├──────────┤ X ├
     ├───┤├───┤   ┌───┐     ┌───┐┌─────┐       │  └───┘└──────┘┌────────┐└─┬─┘
q_1: ┤ Y ├┤ Z ├─X─┤ X ├─────┤ Y ├┤ Sdg ├──■────┼──────────X────┤0       ├──■──
     ├───┤├───┤ │ └─┬─┘┌───┐└─┬─┘├─────┤┌─┴─┐┌─┴─┐┌───┐   │    │  Iswap │     
q_2: ┤ S ├┤ Y ├─X───■──┤ Y ├──■──┤ Sdg ├┤ X ├┤ X ├┤ X ├───X────┤1       ├─────
     └───┘└───┘        └───┘     └─────┘└───┘└───┘└───┘        └────────┘

Convert to qasm via Qiskit: https://docs.quantum.ibm.com/api/qiskit/qasm2#exporting-examples

In [180]:
import qiskit.qasm2
from qiskit.circuit import QuantumCircuit
 
print(qiskit.qasm2.dumps(qiskit_circ))

OPENQASM 2.0;
include "qelib1.inc";
gate iswap q0,q1 { s q0; s q1; h q0; cx q0,q1; cx q1,q0; h q1; }
qreg q[3];
y q[1];
z q[1];
s q[2];
y q[0];
y q[2];
swap q[1],q[2];
x q[0];
cx q[2],q[1];
y q[2];
cy q[2],q[1];
sdg q[2];
sdg q[1];
cx q[1],q[2];
cx q[0],q[2];
z q[0];
x q[2];
sxdg q[0];
swap q[1],q[2];
iswap q[1],q[2];
cx q[1],q[0];


In [181]:
#Now add it to a file for clarity

qiskit.qasm2.dump(qiskit_circ, "random_circuit.qasm")

## QASM Completed ^

## QASM Conversion 

### Imports

In [182]:
import numpy as np
from qutip import about, basis, rand_ket, tensor
from qutip_qip.operations import Measurement
from qutip_qip.operations.gates import gate_sequence_product
from qutip_qip.qasm import read_qasm, print_qasm

### Read in the file to a string

In [183]:
with open(path, 'r') as file:
    qasm_str = file.read()

In [184]:
type(qasm_str)

str

In [185]:
print(qasm_str)

OPENQASM 2.0;
include "qelib1.inc";
gate iswap q0,q1 { s q0; s q1; h q0; cx q0,q1; cx q1,q0; h q1; }
qreg q[3];
y q[1];
z q[1];
s q[2];
y q[0];
y q[2];
swap q[1],q[2];
x q[0];
cx q[2],q[1];
y q[2];
cy q[2],q[1];
sdg q[2];
sdg q[1];
cx q[1],q[2];
cx q[0],q[2];
z q[0];
x q[2];
sxdg q[0];
swap q[1],q[2];
iswap q[1],q[2];
cx q[1],q[0];



In [186]:
from qiskit.circuit.library import SXdgGate

# Decompose the SXdg gate using Qiskit
sxdg_gate = SXdgGate()
decomposition_circuit = sxdg_gate.definition
print(decomposition_circuit.draw())

global phase: 7π/4
   ┌───┐┌───┐┌───┐
q: ┤ S ├┤ H ├┤ S ├
   └───┘└───┘└───┘


### Append basic gate definitions to the string

In [ ]:
#swap gate

lines = qasm_str.splitlines()
for i, line in enumerate(lines):
    if line.strip().startswith('include'):
        insert_idx = i + 1
        break

swap_gate_def = [
    'gate swap a, b{',
    'cx b, a;',
    'cx a, b;',
    'cx b, a;',
    '}'
]

sx_gate_def = [
    'gate sx a {'
    'u1(-pi/2) a;'
    'u2(0,pi) a;'
    'u1(-pi/2) a;'
    '}'
]

lines[insert_idx:insert_idx] = swap_gate_def
lines.insert(insert_idx + 5, '')
lines[insert_idx + 2:insert_idx + 2] = sx_gate_def
qasm_str = '\n'.join(lines)

print(qasm_str)

OPENQASM 2.0;
include "qelib1.inc";
gate swap a, b{
cx b, a;
gate sx a {u1(-pi/2) a;u2(0,pi) a;u1(-pi/2) a;}
cx a, b;
cx b, a;

}
gate iswap q0,q1 { s q0; s q1; h q0; cx q0,q1; cx q1,q0; h q1; }
qreg q[3];
y q[1];
z q[1];
s q[2];
y q[0];
y q[2];
swap q[1],q[2];
x q[0];
cx q[2],q[1];
y q[2];
cy q[2],q[1];
sdg q[2];
sdg q[1];
cx q[1],q[2];
cx q[0],q[2];
z q[0];
x q[2];
sxdg q[0];
swap q[1],q[2];
iswap q[1],q[2];
cx q[1],q[0];


### Review the string using the built-in qasm to qutip conversion tools

In [188]:
qutip_circ = read_qasm(qasm_str, strmode=True)
gate_sequence_product(qutip_circ.propagators())

SyntaxError: QASM: incorrect bracket formatting (<string>)